Image-to-Image tasks apply when an application receives an image and outputs another image. This has various subtasks, including image enhancement (super resolution, low light enhancement, deraining and so on), image inpainting, and more.

This guide will show you how to:
1. Use an image-to-image pipeline for super resolution task,
2. Run image-to-image models for same task without a pipeline.

NB: image-to-image pipeline only supports super resolution task at the time of coding.

# Libraries

In [ ]:
pip install transformers